In [66]:
import pandas as pd

import wmfdata as wmf
from wmfdata.utils import (
    insert_code_toggle,
    pd_display_all,
    print_err
)

In [67]:
insert_code_toggle()

In [3]:
def sql_tuple(i):
    """
    Making an SQL 'tuple', for use in an IN clause, is hard. Doing it manually using 
    `", ".join` requires a lot of messing around with quote marks and escaping. Using the
    string representation of a Python tuple *almost* works, but fails when there's just
    one element, because SQL doesn't accept the trailing comma that Python uses.
    What we really want is the string representation of a Python list, but using parentheses
    instead of brackets. This function turns an iterable into just that.
    """
    # Transform other iterables into lists, raising errors for non-iterables
    if type(i) != list:
        i = [x for x in i]
    
    # Don't return empty SQL tuples, since they cause syntax errors 
    if len(i) == 0:
        return None

    list_repr = repr(i)

    return "(" + list_repr[1:-1] + ")"

The experiment was released to the KaiOS store on 8 April, in version 1.6.0 of the app. Based on the data being collected so far, is the experiment and its data collection running correctly?

In [4]:
experiment_uniques = wmf.presto.run("""
WITH refined AS (
  SELECT
    CAST(FROM_ISO8601_TIMESTAMP(meta.dt) AS DATE) AS date,
    event.user_id AS user_id,
    geocoded_data['country'] AS country,
    REDUCE(
      event.tests, 
      NULL, 
      (i, x) -> IF(x.name = '2021-KaiOS-app-homepage-content-suggestions', x."group", i),
      x -> x
    ) AS experiment_group
  FROM event.inukapageview
  WHERE
    geocoded_data['country'] IN ('Nigeria', 'Pakistan', 'Tanzania', 'Uganda')
    AND year = 2021
    AND month = 4
    AND day >= 8
    AND event.app_version = '1.6.0'
    AND event.is_production
), uniques AS (
  SELECT
    ARBITRARY(country) AS country,
    ARBITRARY(experiment_group) AS experiment_group
  FROM refined
  GROUP BY
    user_id
)
SELECT
  country,
  experiment_group,
  COUNT(*) AS devices
FROM uniques
GROUP BY
  country,
  experiment_group
ORDER BY
  country,
  experiment_group
""")

Based on [my power analysis](T271316#6893769), we expected to accumulate about 12,000 devices as experiment participants over a 6 week experiment.

So far, in roughly 16 days, we have accumulated about 11,500, so we seem to be well ahead of schedule. 

Moreover, these numbers meet two other expectations: the users are split quite evenly between the two groups, and all users from our four participating countries are recorded as being in one of the two groups. 

In [5]:
pd.pivot_table(experiment_uniques, index="country", columns="experiment_group", aggfunc="sum", margins=True)

devices                         
experiment_group control trending-articles    All
country                                          
Nigeria             1234              1188   2422
Pakistan            3050              2943   5993
Tanzania            1296              1292   2588
Uganda               887               887   1774
All                 6467              6310  12777

In [9]:
# Have any of these uniques "crossed oved" between countries or experiment groups?
crossover_uniques = wmf.presto.run("""
WITH refined AS (
  SELECT
    CAST(FROM_ISO8601_TIMESTAMP(meta.dt) AS DATE) AS date,
    event.user_id AS user_id,
    geocoded_data['country'] AS country,
    REDUCE(
      event.tests, 
      NULL, 
      (i, x) -> IF(x.name = '2021-KaiOS-app-homepage-content-suggestions', x."group", i),
      x -> x
    ) AS experiment_group
  FROM event.inukapageview
  WHERE
    geocoded_data['country'] IN ('Nigeria', 'Pakistan', 'Tanzania', 'Uganda')
    AND year = 2021
    AND month = 4
    AND day >= 8
    AND event.app_version = '1.6.0'
    AND event.is_production
)
SELECT
    user_id,
    SET_AGG(country) AS distinct_countries,
    SET_AGG(experiment_group) AS distinct_groups
FROM refined
GROUP BY user_id
HAVING (
    COUNT(DISTINCT country) > 1
    OR COUNT(DISTINCT experiment_group) > 1
)
""")

None of these users was recorded in more than one country (which should be extremely rare). A single one was recorded in both experiment groups. This should never occur, but since it's just one, it won't affect the results.

In [13]:
crossover_uniques.assign(
    user_id=lambda df: df["user_id"].str.slice_replace(0, 16, repl="X"*16)
)

,user_id,distinct_countries,distinct_groups
0,XXXXXXXXXXXXXXXXa768,[Pakistan],"[trending-articles, control]"


We intentionally do not track the names of the pages read together with the user ID, so we cannot check the groupwise reading patterns to verify that the users in the trending articles group are actually seeing the recommendations.

However, we do track the aggregate number of views to specific pages by various client _types_, so we can check the overall reading patterns of KaiOS app users in these countries to verify that there is some increase in pageviews to top recommended articles.

In [17]:
top_recommended_articles = (
    pd.read_csv("trending_articles.csv", parse_dates=["date"])
    .query("date >= '2021-04-08' & date < '2021-04-26' & rank == 1 & country != 'IN'")
    .reset_index(drop=True)
    [["date", "country", "article"]]
    .rename({"date": "trending_date"}, axis="columns")
    .assign(recommended_date=lambda df: df["trending_date"] + pd.DateOffset(days=1))
)

In [18]:
relevant_pages = top_recommended_articles["article"].pipe(sql_tuple)

relevant_views = wmf.spark.run(f"""
SELECT
    DATE(CONCAT_WS('-', year, LPAD(CAST(month AS STRING), 2, '0'), LPAD(CAST(day AS STRING), 2, '0'))) AS `date`,
    country_code AS country,
    page_title,
    SUM(view_count) AS views,
    SUM(IF(user_agent_map['os_family'] = 'KaiOS' AND access_method = 'mobile app', view_count, 0)) AS KaiOS_views
FROM wmf.pageview_hourly
WHERE
    page_title IN {relevant_pages}
    AND country_code IN ('NG', 'PK', 'TZ', 'UG')
    AND agent_type = 'user'
    AND project = 'en.wikipedia'
    AND year = 2021
    AND month = 4
    AND day >= 8
GROUP BY
    page_title,
    country_code,
    year,
    month,
    day
""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [19]:
trending_date_relevant_views = (
    relevant_views
    .rename({
        "views": "trending_date_views", 
        "KaiOS_views": "KaiOS_trending_date_views",
        "date": "trending_date",
        "page_title": "article"
    }, axis="columns")
    .assign(trending_date=lambda df: pd.to_datetime(df["trending_date"]))
)

top_recommended_articles = pd.merge(
    top_recommended_articles,
    trending_date_relevant_views,
    how="left"
)

recommended_date_relevant_views = (
    relevant_views
    .rename({
        "views": "recommended_date_views", 
        "KaiOS_views": "KaiOS_recommended_date_views",
        "date": "recommended_date",
        "page_title": "article"
    }, axis="columns")
    .assign(recommended_date=lambda df: pd.to_datetime(df["recommended_date"]))
)

top_recommended_articles = pd.merge(
    top_recommended_articles,
    recommended_date_relevant_views,
    how="left"
)

There were 72 daily top recommended articles (4 daily for 18 days). Of these, **59** of them had no views on the day they were recommended (as did 62 on them on the day they were trending). Wow.

In [31]:
(
    top_recommended_articles
    [["KaiOS_trending_date_views", "KaiOS_recommended_date_views"]]
    .applymap(lambda x: x == 0)
    .sum()
    .rename("number_of_0_values")
)

KaiOS_trending_date_views       62
KaiOS_recommended_date_views    59
Name: number_of_0_values, dtype: int64

In [46]:
experiment_pageviews = wmf.presto.run("""
WITH refined AS (
    SELECT
        CAST(FROM_ISO8601_TIMESTAMP(meta.dt) AS DATE) AS "date",
        geocoded_data['country_code'] AS country,
        REDUCE(
          event.tests, 
          NULL, 
          (i, x) -> IF(x.name = '2021-KaiOS-app-homepage-content-suggestions', x."group", i),
          x -> x
        ) AS experiment_group,
        event.pageview_token AS pageview_token
    FROM event.inukapageview
    WHERE
        year = 2021
        AND month = 4
        AND day >= 8
        AND geocoded_data['country_code'] IN ('NG', 'PK', 'TZ', 'UG')
        AND CARDINALITY(FILTER(
            event.tests,
            x -> x.name = '2021-KaiOS-app-homepage-content-suggestions'
        )) > 0
        AND event.is_production
        AND NOT event.is_main_page
        AND NOT event.is_search_page
        -- The recommendations will only be shown if the user is using the app in English
        AND wiki = 'enwiki'
)
SELECT
    date,
    country,
    experiment_group,
    COUNT(DISTINCT pageview_token) AS pageviews
FROM refined
GROUP BY
    date,
    country,
    experiment_group
ORDER BY
    date,
    country,
    experiment_group
""")

However, we *are* recording pageviews from the users in this experiment; below are the number of pageviews per bucket per country for the last two full days, as well the number of unique devices generated in each group.

In [52]:
experiment_uniques_2 = wmf.presto.run("""
WITH refined AS (
      SELECT
        CAST(FROM_ISO8601_TIMESTAMP(meta.dt) AS DATE) AS "date",
        geocoded_data['country_code'] AS country,
        REDUCE(
          event.tests, 
          NULL, 
          (i, x) -> IF(x.name = '2021-KaiOS-app-homepage-content-suggestions', x."group", i),
          x -> x
        ) AS experiment_group,
        event.user_id AS user_id
    FROM event.inukapageview
    WHERE
        year = 2021
        AND month = 4
        AND day >= 8
        AND geocoded_data['country_code'] IN ('NG', 'PK', 'TZ', 'UG')
        AND CARDINALITY(FILTER(
            event.tests,
            x -> x.name = '2021-KaiOS-app-homepage-content-suggestions'
        )) > 0
        AND event.is_production
        AND NOT event.is_main_page
        AND NOT event.is_search_page
        AND wiki = 'enwiki'
), uniques AS (
    SELECT
        date,
        ARBITRARY(country) AS country,
        ARBITRARY(experiment_group) AS experiment_group
    FROM refined
    GROUP BY
        user_id,
        date
)
SELECT
    date,
    country,
    experiment_group,
    COUNT(*) AS unique_devices
FROM uniques
GROUP BY
    date,
    country,
    experiment_group
ORDER BY
    date,
    country,
    experiment_group
""")

In [50]:
experiment_pageviews.query("date in ('2021-04-25', '2021-04-26')")

,date,country,experiment_group,pageviews
136,2021-04-25,NG,control,317
137,2021-04-25,NG,trending-articles,331
138,2021-04-25,PK,control,448
139,2021-04-25,PK,trending-articles,478
140,2021-04-25,TZ,control,255
141,2021-04-25,TZ,trending-articles,243
142,2021-04-25,UG,control,148
143,2021-04-25,UG,trending-articles,148
144,2021-04-26,NG,control,587
145,2021-04-26,NG,trending-articles,263


In [65]:
print(
    "Average unique devices per bucket per country per day:",
    int(experiment_pageviews.query("date < '2021-04-27'")["pageviews"].mean().round())
)

Average unique devices per bucket per country per day: 230


In [54]:
experiment_uniques_2.query("date in ('2021-04-25', '2021-04-26')")

,date,country,experiment_group,unique_devices
136,2021-04-25,NG,control,64
137,2021-04-25,NG,trending-articles,73
138,2021-04-25,PK,control,132
139,2021-04-25,PK,trending-articles,121
140,2021-04-25,TZ,control,65
141,2021-04-25,TZ,trending-articles,63
142,2021-04-25,UG,control,43
143,2021-04-25,UG,trending-articles,40
144,2021-04-26,NG,control,73
145,2021-04-26,NG,trending-articles,63


In [64]:
print(
    "Average unique devices per bucket per country per day:",
    int(experiment_uniques_2.query("date < '2021-04-27'")["unique_devices"].mean().round())
)

Average unique devices per bucket per country per day: 58


So, if the trending articles feed is working correctly, on an average day in a given country, all 58 devices in the trending articles group see the feed, but not a single one reads the first article on the list, even though the group views about 230 pages total. That means that either the feed is _not_ working correctly, or the trending articles feed is _completely_ uninteresting to our users.